In [1]:
import os

import pandas as pd

from kmeans_comparator import KMeansSimulator

from os.path import join
from numpy import min, mean, sum

In [2]:
pwd = os.getcwd()
datadir = join(pwd, 'data/')
files = os.listdir(datadir)
filename = 'boston.csv'
file = join(datadir, filename)
num_clusters = [
    5,
]
inits = [
    'random',
    'k-means++',
    'coc',
    'orss',
]

In [3]:
dfs = []
for filename in files[:1]:
    file = join(datadir, filename)
    comparator = KMeansSimulator(
        n_cluster=num_clusters,
        num_exp=20,
        file=file,
    )
    comparator.run_kmeans()
#     print('here')
    df = {}
#     print(comparator.inertia_)
    for num in num_clusters:
        rows = []
        cols = [
            'Method',
            'Average Inertia', 
            'Average Inertia (PCA)',
            'Average Iterations', 
            'Average Iterations (PCA)',
            'Minimum Inertia',
            'Minimum Inertia (PCA)',
            'Time',
            'Time (PCA)',
        ]
        for init in inits:
            row = []
            row.append(init)
            inertias = comparator.inertia_[num][init]
            inertias_pca = comparator.inertia_with_pca_[num][init]
            times = comparator.time_[num][init]
            times_pca = comparator.time_with_pca[num][init]
            iters = comparator.n_iter_[num][init]
            iters_pca = comparator.n_iter_with_pca[num][init]
            row.append(mean(inertias))
            row.append(mean(inertias_pca))
            row.append(mean(iters))
            row.append(mean(iters_pca))
            row.append(min(inertias))
            row.append(min(inertias_pca))
            row.append(mean(times))
            row.append(mean(times_pca))
            rows.append(row)
        df[num] = pd.DataFrame(rows, columns=cols)
        print(df[num])
    dfs.append(df)

168897 <class 'numpy.int64'>


ValueError: Centers not initialized properly